# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

#### --Import Statements--

In [2]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from azureml.train.automl import AutoMLConfig
from azureml.pipeline.steps import AutoMLStep

from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Workspace, Datastore
from azureml.core.dataset import Dataset

from azureml.pipeline.core.run import PipelineRun
from azureml.widgets import RunDetails
from dataprocess import clean_data

#### --Worspace Initialization--

In [3]:
#---------------------------------------------------------------------------------------------------
#1. Initialize Workspace
# from azureml.core import Workspace, Experiment

# ws = Workspace.get(name="quick-starts-ws-135395")
# exp = Experiment(workspace=ws, name="udacity-project3")

# print('Workspace name: ' + ws.name, 
#       'Azure region: ' + ws.location, 
#       'Subscription id: ' + ws.subscription_id, 
#       'Resource group: ' + ws.resource_group, sep = '\n')

# run = exp.start_logging()

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

The dataset consist information of patient that might be having heart disease. The target column has values 0 and 1 which determines if the patients has heart disease.
There are 14 attributes in the dataset as follows:
1. age
2. sex
3. chest pain type
4. resting blood pressure
5. serum cholestoral in mg/dl
6. fasting blood sugar > 120 mg/dl
7. resting electrocardiographic results (values 0,1,2)
8. maximum heart rate achieved
9. exercise induced angina
10. oldpeak = ST depression induced by exercise relative to rest
11. the slope of the peak exercise ST segment
12. number of major vessels (0-3) colored by flourosopy
13. thal: 3 = normal; 6 = fixed defect; 7 = reversable defect
14. target column, 0 = heart disease not present, 1 = heart disease present

After data is pulled, the data is processed for training in the following way:
- Standardization was performed on the column bloodpressure, thalac and cholestrol to save on computational cost.
- Rows with null values were dropped
- The age column was binned into 7 groups of '29-37', '38-45', '46-53', '54-61', '62-69' and '70-77'

After processing the data, the test input output and train input output data was set and fed to training.

In [4]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'automl-exp'

experiment=Experiment(ws, experiment_name)

In [5]:
cpu_cluster_name = "cpu-cluster2"

try: #--- Verify that cluster does not exist already
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
    
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.

Running


#### --Dataset Object--

In [6]:
key = "heart-dataset"
description_text = "Heart Disease dataset"

#-- Dataset Call
if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 
df = dataset.to_pandas_dataframe()
df.describe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.366337,0.683168,0.966997,131.623762,246.264026,0.148515,0.528053,149.646865,0.326733,1.039604,1.399340,0.729373,2.313531,0.544554
std,9.082101,0.466011,1.032052,17.538143,51.830751,0.356198,0.525860,22.905161,0.469794,1.161075,0.616226,1.022606,0.612277,0.498835
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.500000,0.000000,0.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,55.000000,1.000000,1.000000,130.000000,240.000000,0.000000,1.000000,153.000000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.000000,1.000000,2.000000,140.000000,274.500000,0.000000,1.000000,166.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


## --Preprocessing Data--

In [7]:
x_df, y_df = clean_data(df)

   age_bins  sex  cp  trestbps      chol  fbs  restecg   thalach  exang  \
0         4    1   3  0.763956 -0.256334    1        0  0.015443      0   
1         1    1   2 -0.092738  0.072199    0        1  1.633471      0   
2         1    0   1 -0.092738 -0.816773    0        0  0.977514      0   
3         3    1   1 -0.663867 -0.198357    0        1  1.239897      0   
4         4    0   0 -0.663867  2.082050    0        1  0.583939      1   

   oldpeak  slope  ca  thal  
0      2.3      0   0     1  
1      3.5      0   0     2  
2      1.4      2   0     2  
3      0.8      2   0     2  
4      0.6      2   0     2  
0    1
1    1
2    1
3    1
4    1
Name: target, dtype: int64


In [8]:
### Train Test Split
X_train, X_test, Y_train, Y_test = train_test_split(x_df, y_df, test_size=0.30)
df_train = pd.concat([X_train,Y_train], axis=1)
#print(df_train)
ds_train = Dataset.Tabular.register_pandas_dataframe(dataframe= df_train, target= Datastore.get_default(ws), name="train_data")
#dataset_1 = pd.to_Tabular(df_train) #Dataset.Tabular.from_pandas_dataframe(df_train)

Method register_pandas_dataframe: This is an experimental method, and may change at any time.<br/>For more information, see https://aka.ms/azuremlexperimental.


Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/1f2a81a5-a8cb-433c-be2d-c3ae3b12d663/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

The automl settings is configured as follows:
- Experiment timeout minutes determines how long experiment should run. In this case, its 20 minutes the experiment will run.
- Max concurrent iterations determines how many iterations should be running in parallel. We chose 5 iterations for this experiement.
- The primary metric to determine the performance of the runs being used here is AUC weighted.

In [9]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric": 'AUC_weighted'
}

The automl config is configured as follows:
- The compute target is set to the one created
- The task is classification as the target column has values 0 and 1
- Early stopping is enabled
- The path and log path is assigned

In [10]:
automl_config = AutoMLConfig(compute_target=cpu_cluster,
                             task = 'classification',
                             training_data = ds_train,
                             label_column_name = "target",
                             path = './pipeline-project3',
                             enable_early_stopping = True,
                             featurization = 'auto',
                             debug_log = 'automl_errors.log',
                             **automl_settings
                            )

In [11]:
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
automl-exp,AutoML_f4a84f04-fd89-406f-9114-0cecc0be773c,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [12]:
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)
assert(remote_run.get_status() == "Completed")

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
automl-exp,AutoML_f4a84f04-fd89-406f-9114-0cecc0be773c,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+---------------------------------+
|Number of folds                  |
+=================================+
|10                               |
+---------------------------------+

****************************************************************************************************

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: 

        63   StandardScalerWrapper LogisticRegression       0:02:23       0.8804    0.9155
        64   StandardScalerWrapper LogisticRegression       0:00:44       0.8952    0.9155
        65   TruncatedSVDWrapper LinearSVM                  0:00:41       0.8813    0.9155
        66   MaxAbsScaler LogisticRegression                0:00:41       0.8935    0.9155
        67   MaxAbsScaler SGD                               0:00:40       0.9074    0.9155
        68   StandardScalerWrapper LogisticRegression       0:00:42       0.8911    0.9155
        69                                                  0:00:16          nan    0.9155
        70                                                  0:00:15          nan    0.9155
        71    VotingEnsemble                                0:01:23       0.9194    0.9194
        72    StackEnsemble                                 0:02:36       0.9120    0.9194


In [13]:
remote_run.get_output()

(Run(Experiment: automl-exp,
 Id: AutoML_f4a84f04-fd89-406f-9114-0cecc0be773c_71,
 Type: azureml.scriptrun,
 Status: Completed),
 Pipeline(memory=None,
          steps=[('datatransformer',
                  DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                  feature_sweeping_config=None,
                                  feature_sweeping_timeout=None,
                                  featurization_config=None, force_text_dnn=None,
                                  is_cross_validation=None,
                                  is_onnx_compatible=None, logger=None,
                                  observer=None, task=None, working_dir=None)),
                 ('prefittedsoftvotingclassifier',...
                                                                              <azureml.automl.runtime.shared.model_wrappers.SparseNormalizer object at 0x7f5b7c3cba20>),
                                                                             ('linea

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [14]:
automl_best_run = remote_run.get_best_child()

In [15]:
#TODO: Save the best model
model= automl_best_run.register_model(model_name='udacity-automl-project', model_path='./outputs/model.pkl')

In [16]:
temp_model = remote_run.get_output()
temp_model[-1]

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                             <azureml.automl.runtime.shared.model_wrappers.SparseNormalizer object at 0x7f5bf21e58d0>),
                                                                            ('linearsvmwrapper',
                                                                             LinearSVMWrapper(C=0.2682695795279725,
           

In [22]:
from azureml.pipeline.steps import AutoMLStep
model.steps[0]

AttributeError: 'Model' object has no attribute 'steps'

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [25]:
from azureml.core import Model
service_name = 'heart-sklearn-service'

service = Model.deploy(ws, service_name, [model], overwrite=True)
service.wait_for_deployment(show_output=True)

UserErrorException: UserErrorException:
	Message: You must provide an InferenceConfig when deploying a model with model_framework set to Custom. Default environments are only provided for these frameworks: ['Onnx', 'ScikitLearn', 'TensorFlow', 'Multi'].
	InnerException None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "You must provide an InferenceConfig when deploying a model with model_framework set to Custom. Default environments are only provided for these frameworks: ['Onnx', 'ScikitLearn', 'TensorFlow', 'Multi']."
    }
}

TODO: In the cell below, send a request to the web service you deployed to test it.

In [22]:
import json


input_payload = json.dumps({
    'data': X_test,
    'method': 'predict_prova'  # If you have a classification model, you can get probabilities by changing this to 'predict_proba'.
})

output = service.run(input_payload)


NameError: name 'rest_endpoint' is not defined

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
print(output)

In [ ]:
service.delete()